In [7]:
# Paths
import os
from pathlib import Path
import json
import pprint

pp = pprint.PrettyPrinter(indent=4)

DIR = os.path.join(os.getcwd(), 'guideline_repos', 'GoodReportsImport')
GUIDELINES_FP = os.path.join(DIR, 'guidelines.json')
META_FP = os.path.join(DIR, 'meta.txt')
OUTPUT_DIR = Path(DIR).parent.absolute()

with open(GUIDELINES_FP, 'r') as _file:
    GUIDELINES = json.load(_file)

with open(META_FP, 'r') as _file:
    META = json.load(_file)

In [8]:
from build.paths import RepoPaths
from build.file import save_string

def create_dirs(paths):
    if not os.path.exists(paths.dir): # type: ignore
        print(f"making dir: {paths.dir}")
        os.makedirs(paths.dir) # type: ignore
        os.makedirs(paths.partials_dir)
        os.makedirs(paths.items_dir)

def get_paths(dirname):
    return RepoPaths(dirname)

def make_blank(fpath):
    save_string('', fpath)

METADATA = """\
title: The {acronym} guideline for writing a {description}.
pagetitle: The {acronym} guidelines for writing a {description}.
css: 
  - ../../css/styles.css
  - ../../css/index.css"""

# WHY_USE = """\
# Confidently write articles using guidance developed by experts from the research community.

# This guideline won't tell you how to _do_ research, but it will help you describe your work fully, so that all researchers, evidence synthesisers, clinicians, patients, and search-engines can use it.

# Endorsed by thousands of journals, this guideline will help you write and publish easily.
# """

# HOW_TO_USE = """\
# This reporting guideline is best used early:

# * Use a template to write quickly and avoid the blank page
# * Use a checklist to double-check and reassure others that your work is described transparently

# You don't need to write your article in the order the items are presented. For instance, some people prefer to write the methods section first.

# The guideline does not prescribe a rigid format or standardized content. Consider each item and prioritize elements that are most relevant to your study, findings, context, and readership. The guideline order is just a suggestion, you may prefer to report an item in a different place, section, or in a table or figure. If you feel confident that an item is less important to your study, you could report it in an appendix or supplement, but remember these are harder for readers to find, not peer reviewed, and not indexed by search engines.
# """

WHY_USE, HOW_TO_USE = '', ''

def make_metadata(gl_meta, fpath):
    metadata = METADATA.format(acronym=gl_meta['name'], description=gl_meta['for_'])
    save_string(metadata, fpath)


from collections import defaultdict
import yaml

def make_resource_definitions(paths, items):
    #items.sort(key=lambda x: x['number'])
    dd = defaultdict(list)
    sections = []
    last_section_heading = None
    for item in items:
        filename = make_item_filename(item)
        section_heading = item['RG checklist heading 1']
        if section_heading and section_heading!=last_section_heading:
            item_list = []
            sections.append({section_heading: []})
            last_section_heading = section_heading
        list(sections[-1].values())[0].append(filename)
    definition = {'web': sections, 'checklist': sections}
    with open(paths.resource_definitions, 'w') as file_:
        yaml.dump(definition, file_)

import re

def make_item_filename(item):
    id_ = make_item_id(item)
    return f'{id_}.md'

def make_item_id(item):
    display_number = item['display_number']
    id_ = item['RG checklist heading 2']
    id_ = f"{display_number}_{id_}"
    id_ = id_.lower()
    id_ = id_.replace(' ', '_')
    id_ = re.sub(r'\W', '', id_)
    return id_ 

def make_item_files(items, paths):
    for item in items:
        id_ = make_item_id(item)
        _title = item['RG checklist heading 2'] or item['RG checklist heading 1']
        title = f"{item['display_number']}. {_title}"
        title = title.replace('\\:', ':')
        body = make_item_body(item)
        string = f'---\nid: {id_}\ntitle: "{title}"\n---\n{body}'
        path = os.path.join(paths.items_dir, make_item_filename(item))
        save_string(string, path)

def make_item_body(item):
    text = item['RG Checklist text']
    readmore = item['text_long']
    examples = [v for k, v in item.items() if 'example' in k.lower()]
    examples = [example for example in examples if example]
    examples = [example.replace('\n\n', '\n\n> ') for example in examples]
    examples = ['> '+example for example in examples]
    examples = '\n\n'.join(examples)
    body = f"{text}"
    if readmore:
        body += f"\n\n## Read More\n\n{readmore}"
    if examples:
        body += f"\n\n## Examples\n\n{examples}"
    return body


BLANKS = [
    'not_use_for',
    'related_resources',
]

from guideline_repos.GoodReportsImport.dois import DOIS

def make_config(key, meta, paths):
    doi = DOIS[key]
    config = {'id': key, 'doi': doi, 'translations':[]}
    with open(paths.config, 'w') as file_:
        yaml.dump(config, file_)

def make_glossary(paths):
    with open(paths.glossary, 'w') as file_:
        yaml.dump({})

def copy(key):
    meta = META[key]
    items = GUIDELINES[key]
    acronym = meta['name']
    paths = get_paths(key)
    create_dirs(paths)
    make_metadata(meta, paths.metadata)
    save_string(WHY_USE, paths.why_use)
    save_string(HOW_TO_USE, paths.how_to_use)
    save_string(meta['for_medium'].replace('checklist', 'guideline').split('and is based')[0].strip(), paths.use_for)
    make_resource_definitions(paths, items)
    make_item_files(items, paths)
    make_config(key, meta, paths)
    make_glossary(paths)
    for path_attr in BLANKS:
        make_blank(getattr(paths, path_attr))

import subprocess 
activate = 'source venv/bin/activate'
deactivate = f'deactivate'

for key in GUIDELINES.keys():
    copy(key)
    command = f'make create guideline={key}'
    cmd = '; '.join([activate, command, deactivate])
    subprocess.call(cmd, shell=True, executable='/bin/bash')

python -m build.create_resources consort
python -m build.create_resources strobe
python -m build.create_resources strobe-case-control
python -m build.create_resources strobe-cohort
python -m build.create_resources strobe-cross-sectional
python -m build.create_resources prisma
python -m build.create_resources coreq
python -m build.create_resources arrive
python -m build.create_resources prisma-p
python -m build.create_resources tripod
python -m build.create_resources stard
python -m build.create_resources cheers
python -m build.create_resources squire
python -m build.create_resources strega
python -m build.create_resources spirit
python -m build.create_resources care
python -m build.create_resources moose
